In [4]:
import os
import csv
import pandas as pd
import numpy as np

## Analysis on HMDA file only

- todo - this is only the mini because that's all I have on my comp
- todo - write up findings after this is run on all... For a table... what's the takeaway we (Matt) should look at 


In [6]:
hmda = pd.read_csv('../input_data_clean/state_AZ-CA_clean.csv')

# Our tables will contract BMO and BOW to all other banks
def lei_map(lei):
    if lei == 'QOT5WN9RBKQTFRVKEV31':
        return 'Bank of West'
    elif lei == '3Y4U8VZURTYWI1W2K376':
        return 'BMO Harris'
    elif lei == np.nan:
        return np.nan
    else: 
        return 'All Other Banks'

hmda['which_bank'] = hmda['lei'].map(lei_map)

hmda['approved'] = (hmda['action_taken'] == "Loan Originated") | (hmda['action_taken'] == "Application approved but not accepted") 

hmda['application'] = (hmda['action_taken'] == "Loan Originated") | (hmda['action_taken'] == "Application approved but not accepted") | (hmda['action_taken'] == "Application denied")  

print(hmda['loan_purpose'])


C:\Users\twsca\AppData\Local\Temp\ipykernel_63504\2337139823.py:1: DtypeWarning: Columns (50,51,55,56,64,65,68,69,70,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  hmda = pd.read_csv('../input_data_clean/state_AZ-CA_clean.csv')


0                 Home purchase
1                 Home purchase
2                 Home purchase
3                 Home purchase
4                 Home purchase
                   ...         
2967478           Home purchase
2967479             Refinancing
2967480    Cash-out refinancing
2967481             Refinancing
2967482           Home purchase
Name: loan_purpose, Length: 2967483, dtype: object


## Denial rates by product and purpose (aggregate)

In [8]:
# the denial rate = denials/apps
# so restrict to applications

apps = hmda[(hmda['action_taken'] == "Loan Originated") | (hmda['action_taken'] == "Application approved but not accepted") | (hmda['action_taken'] == "Application denied")].copy()

apps['denial'] = apps['action_taken'] == "Application denied"

# Create the pivot table
pivot = apps.pivot_table(index=['loan_type', 'loan_purpose'], columns = ['which_bank'], values='denial', aggfunc='mean')

# add columns for differentials (easier to see)
pivot['diff_BMO'] = pivot['BMO Harris'] - pivot['All Other Banks']
pivot['diff_BOW'] = pivot['Bank of West'] - pivot['All Other Banks']
pivot.style.format('{:,.2f}')

## Avg requested loan size by product and purpose for all applications (aggregate)

In [9]:
hmda.pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='loan_amount', 
                 aggfunc='mean').style.format('{:,.0f}')

## Avg loan size by product and purpose for approved loans (aggregate)

In [11]:
hmda[hmda['approved']].pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='loan_amount', 
                 aggfunc='mean').style.format('{:,.0f}')

## Rate spread by product and purpose (aggregate)

In [13]:
hmda.pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='rate_spread', 
                 aggfunc='mean').style.format('{:,.2f}')

## Median income of applicants by product and purpose (aggregate)

In [14]:
hmda.pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='income', 
                 aggfunc='mean').style.format('{:,.2f}')

## Median income of approved loans by product and purpose (aggregate)

In [15]:
hmda[hmda['approved']].pivot_table(index=['loan_type', 'loan_purpose'], 
                 columns = ['which_bank'], 
                 values='income', 
                 aggfunc='mean').style.format('{:,.2f}')

In [19]:
LTV = hmda[hmda['approved']]['loan_to_value_ratio'].mean()
print(LTV)

257.1455645313406


## Brainstorming: Tables and figures we could make

- State-level versions of the above to see if there is a different set of facts in either...
- Revisit the project descript doc and the whiteboard picture...
- todo... what else?